### Importing the Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator  
import os
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Conv2D

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
tf.__version__ 

'2.15.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [5]:
train_set = train_datagen.flow_from_directory(r'C:\Users\SHREYASH\Downloads\Models\Models\zeeshan  asli prooject\Sign Language to Text using CNN\dataSet\trainingData',
                                              target_size=(128, 128),
                                              batch_size=32,
                                              color_mode='grayscale',
                                              class_mode='categorical')


Found 12845 images belonging to 27 classes.


In [6]:
test_set = test_datagen.flow_from_directory(r'C:\Users\SHREYASH\Downloads\Models\Models\zeeshan  asli prooject\Sign Language to Text using CNN\dataSet\testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [7]:
classifier = Sequential([
    Input(shape=(128, 128, 1)),  # Define the input shape
    Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")  # No need for input_shape here
])

#### Step 1 - Convolution

In [8]:
#classifier.add(tf.keras.layers.Conv2D(filters=32,
 #                                    kernel_size=3, 
  #                                   padding="same", 
   #                                  activation="relu", 
    #                                 input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [9]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [10]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [11]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [12]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [13]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [16]:
classifier.fit(train_set,
                  epochs = 5,
                  validation_data = test_set)

Epoch 1/5


402/402 [==============================] - 351s 866ms/step - loss: 2.5347 - accuracy: 0.2323 - val_loss: 0.9233 - val_accuracy: 0.7144
Epoch 2/5
402/402 [==============================] - 108s 269ms/step - loss: 1.2834 - accuracy: 0.5608 - val_loss: 0.3148 - val_accuracy: 0.8967
Epoch 3/5
402/402 [==============================] - 107s 266ms/step - loss: 0.9231 - accuracy: 0.6803 - val_loss: 0.1737 - val_accuracy: 0.9531
Epoch 4/5
402/402 [==============================] - 107s 266ms/step - loss: 0.7492 - accuracy: 0.7395 - val_loss: 0.1493 - val_accuracy: 0.9627
Epoch 5/5
402/402 [==============================] - 108s 269ms/step - loss: 0.6514 - accuracy: 0.7746 - val_loss: 0.0940 - val_accuracy: 0.9845


#### Saving the Model

In [26]:
# Save model architecture to JSON file
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model architecture saved')

# Save model weights to an H5 file with the required `.weights.h5` extension
classifier.save_weights('model_new.weights.h5')
print('Model weights saved')


Model architecture saved
Model weights saved
